**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [261]:
import numpy as np
import matplotlib.pyplot as plt
import operator
import functools
import itertools

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [367]:
#%%pycodestyle


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X = np.copy(X)
        self.y = np.copy(y)
        self.classes = np.unique(y)

        def what_class(tmp_class):
            res = [i for i in range(len(self.classes))
                   if self.classes[i] == tmp_class]
            return res[0]
        self.y = np.asarray(list(map(what_class, self.y)))

    def dist(x, y):
        return np.linalg.norm(x-y)

    def predict(self, X):
        def predict_one(x):
            dist = np.argsort(np.linalg.norm(self.X-x, axis=1))
            return self.classes[np.argsort(np.bincount(
                self.y[dist[0:self.n_neighbors]],
                minlength=len(self.classes)))[-1]]
        y = np.asarray(list(map(predict_one, X)))
        return y

    def predict_proba(self, X):
        def predict_proba_one(x):
            dist = np.argsort(np.linalg.norm(self.X-x, axis=1))
            return np.bincount(self.y[dist[0:self.n_neighbors]],
                               minlength=len(self.classes)) / self.n_neighbors
        y = np.asarray(list(map(predict_proba_one, X)))
        return y

    def score(self, X, y):
        y1 = self.predict(X)
        y1 = np.equal(y1, y)
        z = np.sum(y1) / y.size
        return z

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [337]:
iris = datasets.load_iris()

In [338]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [339]:
X_train

array([[5.7, 2.8, 4.1, 1.3],
       [6.3, 2.9, 5.6, 1.8],
       [5.5, 4.2, 1.4, 0.2],
       [5.5, 2.3, 4. , 1.3],
       [4.4, 2.9, 1.4, 0.2],
       [5.5, 2.4, 3.8, 1.1],
       [5.6, 2.5, 3.9, 1.1],
       [6.7, 3.1, 4.4, 1.4],
       [5.2, 3.4, 1.4, 0.2],
       [5.5, 2.4, 3.7, 1. ],
       [7.9, 3.8, 6.4, 2. ],
       [7.7, 3. , 6.1, 2.3],
       [5.7, 4.4, 1.5, 0.4],
       [5. , 3.4, 1.5, 0.2],
       [5.6, 2.9, 3.6, 1.3],
       [6.4, 3.2, 4.5, 1.5],
       [5.4, 3.9, 1.3, 0.4],
       [7.2, 3.2, 6. , 1.8],
       [4.6, 3.2, 1.4, 0.2],
       [5.8, 2.6, 4. , 1.2],
       [4.8, 3.4, 1.6, 0.2],
       [6.4, 2.9, 4.3, 1.3],
       [4.9, 2.4, 3.3, 1. ],
       [5.6, 2.8, 4.9, 2. ],
       [6.7, 3. , 5. , 1.7],
       [6.4, 3.1, 5.5, 1.8],
       [7.4, 2.8, 6.1, 1.9],
       [4.8, 3.4, 1.9, 0.2],
       [6.7, 3. , 5.2, 2.3],
       [5.9, 3. , 4.2, 1.5],
       [7.7, 2.6, 6.9, 2.3],
       [6.1, 3. , 4.6, 1.4],
       [6.3, 2.8, 5.1, 1.5],
       [6. , 2.7, 5.1, 1.6],
       [5. , 2

In [340]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [341]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [342]:
my_clf.predict_proba(X_train)

array([[0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 0.5, 0.5],
       [0. , 0.5, 0.5],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. 

In [343]:
my_clf.predict(X_train)

array([1, 2, 0, 1, 0, 1, 1, 1, 0, 1, 2, 2, 0, 0, 1, 1, 0, 2, 0, 1, 0, 1,
       1, 2, 1, 2, 2, 0, 2, 1, 2, 1, 2, 2, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0,
       2, 0, 2, 0, 1, 2, 1, 2, 0, 0, 1, 1, 1, 2, 1, 0, 2, 1, 1, 0, 0, 0,
       0, 0, 1, 2, 1, 2, 1, 1, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2,
       2, 2, 2, 0, 2, 2, 0, 0, 1, 2, 1, 1, 2, 2, 0, 0, 1, 2, 2, 2, 1, 0,
       0, 0, 2, 1, 2, 1, 0, 2, 2, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0,
       2, 2, 0])

In [344]:
clf.predict(X_train)

array([1, 2, 0, 1, 0, 1, 1, 1, 0, 1, 2, 2, 0, 0, 1, 1, 0, 2, 0, 1, 0, 1,
       1, 2, 1, 2, 2, 0, 2, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0,
       1, 0, 2, 0, 1, 2, 1, 2, 0, 0, 1, 1, 1, 2, 1, 0, 2, 1, 1, 0, 0, 0,
       0, 0, 1, 2, 1, 2, 1, 1, 0, 2, 0, 0, 0, 1, 2, 2, 2, 2, 2, 1, 1, 2,
       2, 2, 2, 0, 2, 2, 0, 0, 1, 2, 1, 1, 2, 2, 0, 0, 1, 2, 2, 2, 1, 0,
       0, 0, 2, 1, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0,
       1, 2, 0])

In [345]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [346]:
%time clf.fit(X_train, y_train)

CPU times: user 1.54 ms, sys: 407 µs, total: 1.94 ms
Wall time: 1.37 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [347]:
%time my_clf.fit(X_train, y_train)

CPU times: user 843 µs, sys: 223 µs, total: 1.07 ms
Wall time: 968 µs


In [348]:
%time clf.predict(X_test)

CPU times: user 1.94 ms, sys: 70 µs, total: 2.01 ms
Wall time: 1.69 ms


array([1, 0, 2, 2, 2, 1, 0, 1, 2, 1, 1, 0, 2, 0, 0])

In [349]:
%time my_clf.predict(X_test)

CPU times: user 1.82 ms, sys: 481 µs, total: 2.3 ms
Wall time: 1.51 ms


array([1, 0, 2, 2, 2, 1, 0, 1, 2, 1, 1, 0, 2, 0, 0])

In [350]:
%time clf.predict_proba(X_test)

CPU times: user 1.79 ms, sys: 472 µs, total: 2.26 ms
Wall time: 1.75 ms


array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [351]:
%time my_clf.predict_proba(X_test)

CPU times: user 2.57 ms, sys: 101 µs, total: 2.67 ms
Wall time: 1.87 ms


array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [ ]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [ ]:
data = newsgroups['data']
target = newsgroups['target']

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [ ]:
data_tok =  #realize here
# data_tok should be a list of lists of tokens for each line in data.

In [ ]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.